In [1]:
from dotenv import load_dotenv
from langchain_google_genai import ChatGoogleGenerativeAI
import os

load_dotenv()

model = ChatGoogleGenerativeAI(
    model="models/gemini-2.5-flash",
    google_api_key=os.getenv("GOOGLE_API_KEY")
)

c:\Langgraph\Sequential_workflow\myenv\Lib\site-packages\langchain_core\_api\deprecation.py:26: UserWarning: Core Pydantic V1 functionality isn't compatible with Python 3.14 or greater.
  from pydantic.v1.fields import FieldInfo as FieldInfoV1


In [13]:
from typing import TypedDict
from dotenv import load_dotenv
from langgraph.graph import StateGraph, START, END
class BlogState(TypedDict):

    title: str
    outline: str
    content: str

In [14]:
def create_outline(state: BlogState) -> BlogState:

    # fetch title
    title = state['title']

    # call llm gen outline
    prompt = f'Generate a detailed outline for a blog on the topic - {title}'
    outline = model.invoke(prompt).content

    # update state
    state['outline'] = outline

    return state

In [15]:
def create_blog(state: BlogState) -> BlogState:

    title = state['title']
    outline = state['outline']

    prompt = f'Write a detailed blog on the title - {title} using the follwing outline \n {outline}'

    content = model.invoke(prompt).content

    state['content'] = content

    return state



In [16]:
graph = StateGraph(BlogState)

# nodes
graph.add_node('create_outline', create_outline)
graph.add_node('create_blog', create_blog)

# edges
graph.add_edge(START, 'create_outline')
graph.add_edge('create_outline', 'create_blog')
graph.add_edge('create_blog', END)

workflow = graph.compile()

In [17]:
intial_state = {'title': 'Rise of AI in India'}

final_state = workflow.invoke(intial_state)

print(final_state)

{'title': 'Rise of AI in India', 'outline': 'Here\'s a detailed outline for a blog post on the "Rise of AI in India," designed to be engaging, informative, and comprehensive.\n\n---\n\n## Blog Title Options:\n\n*   **The AI Awakening: How India is Forging its Future with Artificial Intelligence**\n*   **From Silicon Valley to Bengaluru: India\'s Rapid Ascent in the AI Revolution**\n*   **India\'s AI Boom: Unpacking the Drivers, Impact, and Future of Artificial Intelligence**\n*   **AI in the Bharat Story: Transforming Industries and Lives Across India**\n\n---\n\n## Blog Outline: The Rise of AI in India\n\n### I. Introduction (The Hook & Thesis)\n\n*   **A. Catchy Opening Hook:**\n    *   Start with a startling statistic about AI\'s global growth or India\'s tech prowess.\n    *   Paint a picture of AI\'s pervasive influence (e.g., "From personalized recommendations to smart city solutions...").\n    *   Briefly mention the global AI race and India\'s unique position.\n*   **B. Setting